**This notebook is an exercise in the [Feature Engineering](https://www.kaggle.com/learn/feature-engineering) course.  You can reference the tutorial at [this link](https://www.kaggle.com/ryanholbrook/creating-features).**

---


# Introduction #

In this exercise you'll start developing the features you identified in Exercise 2 as having the most potential. As you work through this exercise, you might take a moment to look at the data documentation again and consider whether the features we're creating make sense from a real-world perspective, and whether there are any useful combinations that stand out to you.

Run this cell to set everything up!

In [1]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.feature_engineering_new.ex3 import *

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor


def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score


# Prepare data
df = pd.read_csv("../input/fe-course-data/ames.csv")
X = df.copy()
y = X.pop("SalePrice")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


-------------------------------------------------------------------------------

Let's start with a few mathematical combinations. We'll focus on features describing areas -- having the same units (square-feet) makes it easy to combine them in sensible ways. Since we're using XGBoost (a tree-based model), we'll focus on ratios and sums.

# 1) Create Mathematical Transforms

Create the following features:

- `LivLotRatio`: the ratio of `GrLivArea` to `LotArea`
- `Spaciousness`: the sum of `FirstFlrSF` and `SecondFlrSF` divided by `TotRmsAbvGrd`
- `TotalOutsideSF`: the sum of `WoodDeckSF`, `OpenPorchSF`, `EnclosedPorch`, `Threeseasonporch`, and `ScreenPorch`

In [2]:
X[["WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "Threeseasonporch", "ScreenPorch"]].sum(axis=1)

0       272.0
1       260.0
2       429.0
3         0.0
4       246.0
        ...  
2925    120.0
2926    164.0
2927    112.0
2928    278.0
2929    238.0
Length: 2930, dtype: float64

In [3]:
# YOUR CODE HERE
X_1 = pd.DataFrame()  # dataframe to hold new features

X_1["LivLotRatio"] = X["GrLivArea"] / X["LotArea"]
# X_1["Spaciousness"] = (X["FirstFlrSF"] + X["SecondFlrSF"]) / X["TotRmsAbvGrd"]  or
X_1["Spaciousness"] = (X[["FirstFlrSF", "SecondFlrSF"]].sum(axis=1)) / X["TotRmsAbvGrd"]
X_1["TotalOutsideSF"] = X[["WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "Threeseasonporch", "ScreenPorch"]].sum(axis=1)


# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [4]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

-------------------------------------------------------------------------------

If you've discovered an interaction effect between a numeric feature and a categorical feature, you might want to model it explicitly using a one-hot encoding, like so:

```
# One-hot encode Categorical feature, adding a column prefix "Cat"
X_new = pd.get_dummies(df.Categorical, prefix="Cat")

# Multiply row-by-row
X_new = X_new.mul(df.Continuous, axis=0)

# Join the new features to the feature set
X = X.join(X_new)
```

# 2) Interaction with a Categorical

We discovered an interaction between `BldgType` and `GrLivArea` in Exercise 2. Now create their interaction features.

In [5]:
# My
X.BldgType.value_counts(), X.BldgType

(OneFam      2425
 TwnhsE       233
 Duplex       109
 Twnhs        101
 TwoFmCon      62
 Name: BldgType, dtype: int64,
 0       OneFam
 1       OneFam
 2       OneFam
 3       OneFam
 4       OneFam
          ...  
 2925    OneFam
 2926    OneFam
 2927    OneFam
 2928    OneFam
 2929    OneFam
 Name: BldgType, Length: 2930, dtype: object)

In [6]:
# My
X_test = pd.get_dummies(X.BldgType, prefix="Bldg")
X_test

,Bldg_Duplex,Bldg_OneFam,Bldg_Twnhs,Bldg_TwnhsE,Bldg_TwoFmCon
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
2925,0,1,0,0,0
2926,0,1,0,0,0
2927,0,1,0,0,0
2928,0,1,0,0,0


In [7]:
# My
X_test.mul(X.BldgType, axis=0).head(10)

,Bldg_Duplex,Bldg_OneFam,Bldg_Twnhs,Bldg_TwnhsE,Bldg_TwoFmCon
0,,OneFam,,,
1,,OneFam,,,
2,,OneFam,,,
3,,OneFam,,,
4,,OneFam,,,
5,,OneFam,,,
6,,,,TwnhsE,
7,,,,TwnhsE,
8,,,,TwnhsE,
9,,OneFam,,,


In [8]:
# YOUR CODE HERE
# One-hot encode BldgType. Use `prefix="Bldg"` in `get_dummies`
# X_2 = ...
X_2 = pd.get_dummies(X.BldgType, prefix="Bldg") 
# Multiply
# X_2 = X_2.mul(..., axis = 0)
X_2 = X_2.mul(X.GrLivArea, axis=0)

# Check your answer
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [9]:
X_2.head(10)

,Bldg_Duplex,Bldg_OneFam,Bldg_Twnhs,Bldg_TwnhsE,Bldg_TwoFmCon
0,0.0,1656.0,0.0,0.0,0.0
1,0.0,896.0,0.0,0.0,0.0
2,0.0,1329.0,0.0,0.0,0.0
3,0.0,2110.0,0.0,0.0,0.0
4,0.0,1629.0,0.0,0.0,0.0
5,0.0,1604.0,0.0,0.0,0.0
6,0.0,0.0,0.0,1338.0,0.0
7,0.0,0.0,0.0,1280.0,0.0
8,0.0,0.0,0.0,1616.0,0.0
9,0.0,1804.0,0.0,0.0,0.0


In [10]:
# Lines below will give you a hint or solution code
# q_2.hint()
# q_2.solution()

# 3) Count Feature

Let's try creating a feature that describes how many kinds of outdoor areas a dwelling has. Create a feature `PorchTypes` that counts how many of the following are greater than 0.0:

```
WoodDeckSF
OpenPorchSF
EnclosedPorch
Threeseasonporch
ScreenPorch
```

In [11]:
X[["WoodDeckSF",
"OpenPorchSF",
"EnclosedPorch",
"Threeseasonporch",
"ScreenPorch"]]

,WoodDeckSF,OpenPorchSF,EnclosedPorch,Threeseasonporch,ScreenPorch
0,210.0,62.0,0.0,0.0,0.0
1,140.0,0.0,0.0,0.0,120.0
2,393.0,36.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,212.0,34.0,0.0,0.0,0.0
...,...,...,...,...,...
2925,120.0,0.0,0.0,0.0,0.0
2926,164.0,0.0,0.0,0.0,0.0
2927,80.0,32.0,0.0,0.0,0.0
2928,240.0,38.0,0.0,0.0,0.0


In [12]:
X_3 = pd.DataFrame()

# YOUR CODE HERE
porch_columns =["WoodDeckSF","OpenPorchSF","EnclosedPorch","Threeseasonporch","ScreenPorch"]
# Създаване на колоната "PorchTypes", като се броят, колко от колоните имат стойности > 0
X_3["PorchTypes"] = (X[porch_columns] > 0).sum(axis=1)

# Check your answer
q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [13]:
X_3["PorchTypes"]

0       2
1       2
2       2
3       0
4       2
       ..
2925    1
2926    1
2927    2
2928    2
2929    2
Name: PorchTypes, Length: 2930, dtype: int64

### Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

In [14]:
# My
# Lines below will give you a hint or solution code
#q_3.hint() 
#q_3.solution()

# 4) Break Down a Categorical Feature

`MSSubClass` describes the type of a dwelling(жилище):

In [15]:
df.MSSubClass.unique()

array(['One_Story_1946_and_Newer_All_Styles', 'Two_Story_1946_and_Newer',
       'One_Story_PUD_1946_and_Newer',
       'One_and_Half_Story_Finished_All_Ages', 'Split_Foyer',
       'Two_Story_PUD_1946_and_Newer', 'Split_or_Multilevel',
       'One_Story_1945_and_Older', 'Duplex_All_Styles_and_Ages',
       'Two_Family_conversion_All_Styles_and_Ages',
       'One_and_Half_Story_Unfinished_All_Ages',
       'Two_Story_1945_and_Older', 'Two_and_Half_Story_All_Ages',
       'One_Story_with_Finished_Attic_All_Ages',
       'PUD_Multilevel_Split_Level_Foyer',
       'One_and_Half_Story_PUD_All_Ages'], dtype=object)

You can see that there is a more general categorization described (roughly) by the first word of each category. Create a feature containing only these first words by splitting `MSSubClass` at the first underscore `_`. (Hint: In the `split` method use an argument `n=1`.)

In [16]:
X_4 = pd.DataFrame()

# YOUR CODE HERE
# MSClass
X_4["MSClass"] = X["MSSubClass"].str.split("_", n=1).str[0]
# or
# X_4["MSClass"] = df.MSSubClass.str.split("_", n=1, expand=True)[0]

# Check your answer
q_4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [17]:
# Lines below will give you a hint or solution code
# q_4.hint()
# q_4.solution()

In [18]:
X_4

,MSClass
0,One
1,One
2,One
3,One
4,Two
...,...
2925,Split
2926,One
2927,Split
2928,One


# 5) Use a Grouped Transform

The value of a home often depends on how it compares to typical homes in its neighborhood. Create a feature `MedNhbdArea` that describes the *median* of `GrLivArea` grouped on `Neighborhood`.

In [19]:
X[["GrLivArea" , "Neighborhood"]].groupby("Neighborhood").median()
# Средните(median) стойности на "GrLivArea" по групи

,GrLivArea
Neighborhood,
Bloomington_Heights,1455.5
Blueste,1118.0
Briardale,1092.0
Brookside,1231.0
Clear_Creek,1694.0
College_Creek,1504.0
Crawford,1648.0
Edwards,1195.5
Gilbert,1560.0


In [20]:
X_5 = pd.DataFrame()

# YOUR CODE HERE
# X_5["MedNhbdArea"] = X.groupby("Neighborhood").GrLivArea.median()  -> wrong because only 28 rows
# correct
X_5["MedNhbdArea"] = X.groupby("Neighborhood")["GrLivArea"].transform("median")

# Check your answer
q_5.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [21]:
X_5

,MedNhbdArea
0,1200.0
1,1200.0
2,1200.0
3,1200.0
4,1560.0
...,...
2925,1282.0
2926,1282.0
2927,1282.0
2928,1282.0


In [22]:
# Lines below will give you a hint or solution code
# q_5.hint()
# q_5.solution()

Now you've made your first new feature set! If you like, you can run the cell below to score the model with all of your new features added:

In [23]:
X_new = X.join([X_1, X_2, X_3, X_4, X_5])
score_dataset(X_new, y), score_dataset(X, y)

(0.13865658070461215, 0.14284209506143358)

# Keep Going #

[**Untangle spatial relationships**](https://www.kaggle.com/ryanholbrook/clustering-with-k-means) by adding cluster labels to your dataset.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/feature-engineering/discussion) to chat with other learners.*